# This notebook shows how to use the frequential class to analyse real data.

2025 notebook from the frquency analysis of Computing group

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pyheatmy import *
from pyheatmy.frequency import frequency_analysis

ModuleNotFoundError: No module named 'pyheatmy'

In [ ]:
df_temp = pd.read_csv("../dataAnalysis/raw_data/point36_12_07_16/point36_temperature_cleaned.csv")
df_press = pd.read_csv("../dataAnalysis/raw_data/point36_12_07_16/point36_pression_cleaned.csv")

In [ ]:
temp10 = df_temp["Temp_profondeur_10"]
temp20 = df_temp["Temp_profondeur_20"]
temp30 = df_temp["Temp_profondeur_30"]
temp_riv = df_press["temperature_stream"]
dates = df_press["dates"]

depths = [0, 0.1, 0.2, 0.3]

signals = np.array([temp_riv, temp10, temp20, temp30])

In [ ]:
signals.shape

Quick plot for overview

In [ ]:
plt.figure(figsize=(6.4, 4.8))
cutoff = 100
for i in range(signals.shape[0]):
    plt.plot(dates[cutoff:], signals[i, cutoff:], label=f'Sensor at depth {depths[i]} m')

# Do not plot every date time
plt.xticks(dates[::100], rotation=90)
plt.xlabel("Dates")
plt.ylabel("Temperature °C")

In [ ]:
import pandas as pd
import numpy as np

# colonne exemple : "12-07-2016 15:00"
# préciser le format accélère et évite les ambiguïtés
dates = pd.to_datetime(df_press['dates'],
                       format="%d-%m-%Y %H:%M",
                       dayfirst=True,
                       errors='coerce')

# diagnostics rapides
print("dtype:", dates.dtype)
print("NaT count:", dates.isna().sum())
print(dates.head())

# si certaines lignes sont NaT, on supprime les lignes invalides et on aligne les signaux
mask = dates.notna()
dates = dates[mask].reset_index(drop=True)

# # si vos signaux sont des Series (ou np.array rows = sensors), appliquez le même masque :
# signals = np.array([temp_riv, temp10, temp20, temp30])  # shape (n_sensors+1, n_times)
# signals = signals[:, mask.values]                       # garder les colonnes valides

# # appeler la recherche de périodes en passant explicitement dates (numpy datetime64 OK)
# dates_np = dates.to_numpy()   # dtype datetime64[ns]
# dominant_periods_days, dominant_freqs, dominant_amps = fa.find_dominant_periods(dates_np, signals, draw=True)

# Convert date

In [ ]:
fa = frequency_analysis()
depth_sensors = depths
river = signals[0, :]

signals = signals[:, cutoff:]
dates = dates[cutoff:]

# Plotting check figures
fa.fft_sensors(dates, signals, depth_sensors)
fa.plot_temperatures(dates, signals, depth_sensors)

# Finding dominant periods
dominant_periods_days, dominant_freqs, dominant_amps = fa.find_dominant_periods(signals, river, draw=True)

# Estimate a and b for these dominant periods
depths = np.array(depth_sensors)
a_est, a_R2 = fa.estimate_a(dates, signals, depths, dominant_periods_days, verbose=True, draw=True)
b_est, b_R2 = fa.estimate_b(dates, signals, depths, dominant_periods_days, verbose=True, draw=True)

# Checking constants
fa.check_constantes(a_est, b_est)

kappa_e, v_t = fa.perform_inversion(a_est, b_est, dominant_periods_days, verbose=True)